# Datahandling in graphs

In [ ]:
import torch
from torch_geometric.data import Data

In [1]:
edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)

NameError: name 'torch' is not defined